# Shibuya Scramble (.b3dm -> .glb) extraction (Kaggle Notebook)

このノートブックは PLATEAU の渋谷区データセット（plateau-13113-shibuya-ku-2023）をインストールし、3D Tiles（.b3dm）内に埋め込まれた glb をスクランブル交差点付近の範囲で抽出して ZIP 化するためのものです。

事前準備（Kaggle）:
- Notebook の右上「Settings」→ Internet を ON にしてください。
- ディスク容量には制限があります。必要に応じて bbox を狭めてください。

使い方:
1. セルを上から順に実行します。
2. 実行完了後、`/kaggle/working/output_glb.zip` をダウンロードしてください（Kaggle の右ペイン → Output）。

必要なら ZIP をこちらにアップロードしてください。私の方で中身を確認し、不要なモデルの削除や結合・gltf 展開を行います。

In [ ]:
# 1) 必要パッケージをインストール
!pip install -q 'plateaukit[all]' tqdm trimesh pygltflib

In [ ]:
# 2) PLATEAU データセットをインストール / prebuild（時間がかかる場合があります）
# ※ すでにインストール済みの場合はスキップされます
!plateaukit install plateau-13113-shibuya-ku-2023 -y
!plateaukit prebuild plateau-13113-shibuya-ku-2023 -t bldg -t tran -t brid -y

In [ ]:
# 3) tileset.json を検索し、指定 bbox に交差するタイルから .b3dm 内蔵 glb を抽出して zip 化
import json, struct, shutil
from pathlib import Path
from plateaukit import load_dataset

DATASET_ID = "plateau-13113-shibuya-ku-2023"
# スクランブル交差点周辺の bbox（必要に応じて調整）: lon_min, lat_min, lon_max, lat_max
TARGET_BBOX = (139.6996, 35.6588, 139.7014, 35.6602)

OUT_DIR = Path('/kaggle/working/output_glb')
ZIP_PATH = Path('/kaggle/working/output_glb.zip')
OUT_DIR.mkdir(parents=True, exist_ok=True)

def bbox_intersect(region, target_bbox):
    west, south, east, north = region[0], region[1], region[2], region[3]
    lon_min, lat_min, lon_max, lat_max = target_bbox
    if lon_max < west or lon_min > east: return False
    if lat_max < south or lat_min > north: return False
    return True

def extract_b3dm_to_glb_bytes(data: bytes):
    if len(data) < 28:
        raise ValueError('b3dm too small')
    magic = data[0:4].decode('ascii', errors='ignore')
    if magic != 'b3dm':
        raise ValueError('not b3dm')
    ftJsonLen = struct.unpack_from('<I', data, 12)[0]
    ftBinLen = struct.unpack_from('<I', data, 16)[0]
    btJsonLen = struct.unpack_from('<I', data, 20)[0]
    btBinLen = struct.unpack_from('<I', data, 24)[0]
    glb_offset = 28 + ftJsonLen + ftBinLen + btJsonLen + btBinLen
    byteLength = struct.unpack_from('<I', data, 8)[0]
    end = byteLength if (byteLength <= len(data) and byteLength > glb_offset) else len(data)
    return data[glb_offset:end]

print('Loading dataset object via plateaukit...')
ds = load_dataset(DATASET_ID)
dataset_root = None
for attr in ('root','path','root_path','_root'):
    if hasattr(ds, attr):
        candidate = getattr(ds, attr)
        if isinstance(candidate, str):
            dataset_root = Path(candidate).resolve(); break
        if isinstance(candidate, Path):
            dataset_root = candidate.resolve(); break

if dataset_root is None:
    # fallback: search common locations
    home = Path.home()
    possible = [home / '.plateaukit', home / '.cache' / 'plateaukit', Path.cwd()]
    found = None
    for base in possible:
        if not base.exists():
            continue
        for p in base.rglob('tileset.json'):
            if DATASET_ID in str(p):
                found = p.parent
                break
        if found:
            dataset_root = found
            break

if dataset_root is None:
    # brute force search under home (may be slow)
    for p in Path.home().rglob('tileset.json'):
        if DATASET_ID in str(p):
            dataset_root = p.parent
            break

if dataset_root is None:
    raise SystemExit('dataset root not found. Check plateaukit install / plateaukit info')

print('Detected dataset root:', dataset_root)
tileset_paths = list(dataset_root.rglob('tileset.json'))
print('Found tileset.json count:', len(tileset_paths))

extracted = 0
processed = set()

def process_tile(tile, base_path):
    global extracted
    bv = tile.get('boundingVolume', {})
    region = bv.get('region')
    intersects = True
    if region:
        intersects = bbox_intersect(region, TARGET_BBOX)
    content = tile.get('content')
    if intersects and content:
        uri = content.get('uri') or content.get('url')
        if uri:
            absolute = (base_path / uri).resolve()
            if absolute.exists():
                if absolute.suffix.lower() == '.b3dm' and absolute not in processed:
                    processed.add(absolute)
                    try:
                        data = absolute.read_bytes()
                        glb = extract_b3dm_to_glb_bytes(data)
                        outp = OUT_DIR / (absolute.stem + '.glb')
                        outp.write_bytes(glb)
                        print('Extracted:', outp)
                        extracted += 1
                    except Exception as e:
                        print('Error extracting', absolute, e)
                elif absolute.suffix.lower() in ('.glb', '.gltf') and absolute not in processed:
                    processed.add(absolute)
                    dst = OUT_DIR / absolute.name
                    shutil.copy2(absolute, dst)
                    print('Copied:', dst)
                    extracted += 1
    for child in tile.get('children', []):
        process_tile(child, base_path)

for ts in tileset_paths:
    try:
        js = json.loads(ts.read_text(encoding='utf-8'))
    except Exception as e:
        print('Parse error', ts, e); continue
    root = js.get('root')
    if root:
        process_tile(root, ts.parent)
    else:
        for t in js.get('tiles', []):
            process_tile(t, ts.parent)

print('Extracted count:', extracted)

if ZIP_PATH.exists():
    ZIP_PATH.unlink()
shutil.make_archive(str(ZIP_PATH.with_suffix('')), 'zip', root_dir=OUT_DIR)
print('Zipped to:', ZIP_PATH)

print('\nDone. Download /kaggle/working/output_glb.zip from the Notebook Output pane.')

In [ ]:
# 4) 出力フォルダの中身確認（任意）
!ls -la /kaggle/working/output_glb || true
!ls -la /kaggle/working/output_glb.zip || true

実行後に `output_glb.zip` をここにアップロードしてください。私が中身を確認して、渋谷センター街・スクランブル交差点周辺だけを抽出／結合／gltf 展開などを行います。必要な出力形式（single .glb / unpacked .gltf + textures / 個別 .glb をそのまま）を教えてください。